In [1]:
%tensorflow_version 2.x

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-11-24 05:36:37--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-11-24 05:36:37--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc40d7609a8b0002c6309df3f702.dl.dropboxusercontent.com/zip_download_get/AnJBI8DSE_8daEqBmpqf_U-b9fblZ_EQan7IBc6pHP2WO2iSPh352CI0GpgWFNsFC84RBIg6oZtx-8k-d2O2QHMq359SV-5s4ZwoZWVM4Ebv2g [following]
--2020-11-24 05:36:37--  https://uc40d7609a8b0002c6309df3f702.dl.dropboxusercontent.com/zip_download_get/AnJBI8DSE_8daEqBmpqf_U-b9fblZ_EQan7IBc6pHP2WO2iSPh352CI0GpgWFNsFC84RBIg6oZtx-8k-d2O2

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [4]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

In [5]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [6]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [7]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , Is female sexism or racism against men a positive sexism? I mean on TV is widely used and most men even accept it                                                    1
Can low blood sugar cause bone marrow problem?                                                                                                                       1
What will be the difference of marks from general open category to SC/St category?                                                                                   1
What should I do if I am falsely accused of cheating and let off with a warning and threat to L4 even for anything silly in future at MIT, Manipal?                  1
How many seats are there in West Bengal for NEET PG?                                                                                                                 1
                                                                                                                                                     

In [8]:
filter.drop([1306122,1306123],0,inplace=True)

In [9]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [10]:
filter.drop(['PK'],1,inplace=True)

In [11]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [12]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [13]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-11-24 05:37:23--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-11-24 05:37:23--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-11-24 05:37:23--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [14]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [15]:
!rm /content/glove.42B.300d.zip

In [17]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [18]:
len(embeding_index['provide'])

300

In [19]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [20]:
len(embeding_index)

1917494

In [21]:
X=filter['question_text']
Y=filter['target']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [23]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53291006 8.096462  ]


In [24]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329100573355543, 1: 8.096462000991817}

In [25]:
x_train

142346     What is the best thing I need to do to join th...
933197     How do I find latest contest which is going to...
893844     Is something like the 1995 Norwegian rocket in...
696806     I am planning to study in Amsterdam at UVA fro...
591187     Human Ecology is fascinating, how can I study ...
                                 ...                        
1297831    Has Quora removed the 'hide' option in message...
246767     What does the chariot mean as a relationship o...
253587     Amar has nine friends in how many ways can be ...
1292671    Does Massachusetts have an equivalent law to C...
432534           How do I again lock bootloader on honor 5x?
Name: question_text, Length: 1044897, dtype: object

In [26]:
from nltk import word_tokenize

In [27]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [28]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [29]:
np.percentile(sent_lens,99)

44.0

In [30]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [31]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'get': 32,
 'when': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'was': 48,
 'did': 49,
 'we': 50,
 'any': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'than': 70,
 'time': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'use': 75,
 'someone': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [32]:
vocab_size

196077

In [33]:
seq_train_matrix.shape

(1044897, 44)

In [34]:
seq_train

[[2,
  3,
  1,
  34,
  185,
  8,
  120,
  5,
  11,
  5,
  371,
  1,
  14004,
  7136,
  1240,
  6,
  107,
  395,
  6063],
 [9, 11, 8, 91, 1478, 5403, 30, 3, 191, 5, 32, 2964, 6, 53408, 257, 13, 2288],
 [3, 194, 42, 1, 12283, 7417, 3379, 3198, 129, 108],
 [8,
  80,
  1075,
  5,
  186,
  6,
  6486,
  43,
  24089,
  27,
  4742,
  210,
  12122,
  2,
  3,
  1,
  34,
  79,
  5,
  91,
  4,
  247,
  5,
  180,
  12,
  39,
  4,
  429,
  7,
  3642,
  3755],
 [235, 10124, 3, 9032, 9, 15, 8, 186, 17, 6, 145],
 [16, 3, 3877, 153, 5987],
 [2, 12, 38, 7, 64511, 88211, 298, 1019],
 [15, 50, 16621, 29, 2666, 46, 4499, 263, 6654],
 [2, 12, 38, 7, 1, 3144, 34, 941, 89, 174],
 [2, 3, 12284],
 [9, 8, 32, 110, 6, 17, 168, 6, 943],
 [2, 146, 31, 25, 673, 446, 7, 144, 534, 4, 245],
 [2, 1142, 7, 1, 88212, 5575, 1866, 11731, 10, 14559],
 [49,
  51,
  33274,
  27,
  2245,
  525,
  89,
  657,
  728,
  10,
  842,
  21,
  4692,
  35489,
  149,
  95,
  229,
  109],
 [16, 3, 1, 5871, 6, 15776, 52, 88213],
 [2, 3, 1226

In [35]:
x_train.shape

(1044897,)

In [36]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [37]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [38]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [39]:
 len(embedding_matrix)

196078

In [40]:
embedding_matrix.shape

(196078, 300)

In [41]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [-0.38356   , -0.029476  ,  0.13224   , ..., -0.18828   ,
         0.44599   , -0.23728   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.47024   ,  0.17335001, -0.28360999, ...,  0.50713998,
        -0.21028   , -0.08522   ]])

In [42]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [43]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58823400  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [44]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [45]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [46]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [47]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2979 - accuracy: 0.8715
Epoch 00001: val_loss improved from inf to 0.24095, saving model to /content/weights-01-0.2409.h5
1045/1045 [==============================] - 191s 183ms/step - loss: 0.2979 - accuracy: 0.8715 - val_loss: 0.2409 - val_accuracy: 0.9045
Epoch 2/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2518 - accuracy: 0.8929
Epoch 00002: val_loss improved from 0.24095 to 0.22878, saving model to /content/weights-02-0.2288.h5
1045/1045 [==============================] - 195s 186ms/step - loss: 0.2518 - accuracy: 0.8929 - val_loss: 0.2288 - val_accuracy: 0.9040
Epoch 3/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2287 - accuracy: 0.9006
Epoch 00003: val_loss improved from 0.22878 to 0.21188, saving model to /content/weights-03-0.2119.h5
1045/1045 [==============================] - 195s 187ms/step - loss: 0.2287 - accuracy: 0.9006 - val_loss: 0.2119 - val_accuracy: 

In [50]:
best_model = load_model('/content/weights-10-0.1957.h5')

In [51]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58823400  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [52]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9616336198632073

In [53]:
best_model.save('./spam_filter_model_quora_questions.h5')